In [53]:
##RAG pipeline With vector Database

In [113]:
## Data Imgestion
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain_community.document_loaders import TextLoader

loader=TextLoader("../requirements.txt")
document=loader.load()
document

[Document(metadata={'source': '../requirements.txt'}, page_content='groq\nlangchain\nstreamlit\npython-dotenv\npyPDF2\n\n-e .')]

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()
API_KEY = os.getenv("API_KEY")
print("-------->",API_KEY)
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY") 


RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

--------> 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


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
import os 

text = """LangChain is a framework for developing applications powered by large language models (LLMs). 
It provides tools to manage, query, and retrieve relevant data from documents efficiently."""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks=text_splitter.split_text(text)
chunk_documents = [Document(page_content=chunk) for chunk in chunks]
print("-------------->",chunk_documents)



--------------> [Document(metadata={}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs). \nIt provides tools to manage, query, and retrieve relevant data from documents efficiently.')]


In [ ]:
from groq import Groq
from langchain_groq import ChatGroq
from dotenv import load_dotenv 
import os
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain


load_dotenv()

API_KEY = os.getenv("API_KEY")

client = ChatGroq(api_key=API_KEY, model="llama3-70b-8192")


clients = Groq(api_key=API_KEY)


TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""


with open("../src/Response.json","r") as f:
    RESPONSE_JSON=json.load(f)
    
    
    
quiz_generation_prompt=ChatPromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    messages=[ 
        {"role": "user", "content": TEMPLATE}
    ]
)


quiz_chain=LLMChain(llm=client,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

     
     
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


quiz_evaluation_prompt=ChatPromptTemplate(
    input_variables=["subject","quiz"],
    messages=[ 
        {"role": "user", "content": TEMPLATE2}
    ]
)

review_chain=LLMChain(llm=client,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain] , input_variables=["text","number","subject","tone","RESPONSE_JSON"] , output_variables=["quiz","review"],verbose=True)

PATH="../src/data.txt"

with open(PATH,"r") as file:
   TEXT = file.read()
   
   
generate_evaluate_chain

TEXT
NUMBER=20
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON
 

def call_groq(prompt):
    response = clients.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an expert MCQ creator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=3000
    )
    return response.choices[0].message.content



def generate_quiz_prompt(text, number, subject, tone, response_json):
    return TEMPLATE.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        RESPONSE_JSON=json.dumps(response_json)
    ) 

quiz_prompt = generate_quiz_prompt(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)
mcq_response = call_groq(quiz_prompt) 

print(mcq_response)




Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24
Python-dotenv could not parse statement starting at line 25
Python-dotenv could not parse statement starting at line 26
Python-dotenv could not parse statement starting at line 29


{"1": {"mcq": "What is biology?", "options": {"a": "The study of non-living things", "b": "The scientific study of life", "c": "The study of the universe", "d": "The study of chemistry"}, "correct": "b"}, 
"2": {"mcq": "What is the broad scope of biology?", "options": {"a": "It only deals with plants", "b": "It only deals with animals", "c": "It is a natural science with a broad scope", "d": "It only deals with microorganisms"}, "correct": "c"}, 
"3": {"mcq": "What are all organisms made up of?", "options": {"a": "Tissues", "b": "Cells", "c": "Molecules", "d": "Genes"}, "correct": "b"}, 
"4": {"mcq": "What is hereditary information encoded in?", "options": {"a": "Cells", "b": "Genes", "c": "Tissues", "d": "Molecules"}, "correct": "b"}, 
"5": {"mcq": "What is the major theme that explains the unity and diversity of life?", "options": {"a": "Energy processing", "b": "Evolution", "c": "Genetics", "d": "Ecology"}, "correct": "b"}, 
"6": {"mcq": "Why is energy processing important to life?"